In [1]:
import h5py
import numpy as np
from scipy import stats
from scipy import sparse
from statsmodels.stats.multitest import multipletests
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [19]:
f = h5py.File('../Yang/Lung_30/Feature_selection/P_D_mat.h5','r')

In [52]:
data = f['data'][:]
genes=f[".data_dimnames"]['1'][:]
samples=f[".data_dimnames"]['2'][:]
len(samples),len(genes),data.shape

(2000, 15150, (2000, 15150))

In [53]:
genes = np.array([g.decode('utf-8') for g in genes])

In [54]:
N = np.asarray(range(data.shape[1]-1))
genes[N]

array(['AL627309.1', 'AP006222.2', 'RP11-206L10.9', ..., 'RP11-114G22.1',
       'SPRR2E', 'KRT24'], dtype='<U19')

In [55]:
A = data[:,N]  ## independent columns
b = data[:,-1]    ## target column i.e region class

In [56]:
## apply SelectKBest class to extract top 20 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(A,b)

In [65]:
dfscores = pd.DataFrame(fit.scores_)
dfpvalues = pd.DataFrame(fit.pvalues_)
dfcolumns = pd.DataFrame(genes)

#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores,dfpvalues],axis=1)
featureScores.columns = ['genes','Score','Pvalue']  ## naming the dataframe columns
print(featureScores.nlargest(60,'Score'))  ## print 10 best features

          genes          Score  Pvalue
7838    SCGB1A1  122859.346309     0.0
4181    SCGB3A1   15037.399687     0.0
12304      CAPS   10396.613648     0.0
11938    BPIFA1    9723.496146     0.0
13636    MT-ND3    9434.896114     0.0
903      S100A9    9254.162628     0.0
9656   HSP90AA1    7952.238427     0.0
4393     HSPA1A    7668.269629     0.0
909      S100A2    5853.675658     0.0
9823        B2M    5818.370092     0.0
13630    MT-ND2    5285.865538     0.0
11773  SERPINB3    5105.158910     0.0
8941     DYNLL1    4385.034201     0.0
6996     TUBB4B    4374.802037     0.0
12031      SLPI    4371.796465     0.0
12106  C20orf85    3880.182290     0.0
10593     TPPP3    3705.672004     0.0
13581      TFF3    3660.338930     0.0
6548    C9orf24    3642.726678     0.0
9594      CALM1    3559.944346     0.0
11008   ALDH3A1    3542.210449     0.0
4394     HSPA1B    3274.709667     0.0
13638    MT-ND4    3016.600803     0.0
10969       UBB    2951.333290     0.0
6862       LCN2    2940.5

In [60]:
featureScores.to_csv('../Yang/Lung_30/Feature_selection/chi2.csv',index=False)